### Setup / Preparations

Load Libraries

In [1]:
%load_ext autoreload
%autoreload 2


from deepgreen import *
import pandas as pd
import json
from dotenv import load_dotenv
import os

from sentinelhub.api.catalog import SentinelHubCatalog

from sentinelhub import (
    SHConfig,
    CRS,
    BBox,
    BBoxSplitter,
    CustomGridSplitter,
    OsmSplitter,
    TileSplitter,
    UtmGridSplitter,
    UtmZoneSplitter,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    read_data,
    bbox_to_dimensions,
)
from utils import plot_image

Specify Project Paths

In [2]:
path_data = "data"
path_images = "images"

Initialize Sentinelhub Modules

In [3]:
# Init Config
load_dotenv()

CONFIG = SHConfig()
# TODO: Gaht nöd
CONFIG.sh_client_id = os.environ.get("CLIENT_ID")
CONFIG.sh_client_secret = os.environ.get("CLIENT_SECRET")
# Init Catalog
CATALOG = SentinelHubCatalog(config=CONFIG)

Define Area(s) of Intrest

In [4]:
# AOI for Switzerland (100% correct)
AOI_CH = BBox(bbox = (5.956302777777778, 45.81803055555556, 10.491944444444444, 47.80826388888889), 
              crs = CRS.WGS84)
# AOI for Zürich (estimate)
AOI_ZH = BBox(bbox = (8.396073, 47.255932, 8.880802, 47.562627), 
              crs = CRS.WGS84)
# AOI for Winterthur (estimate)
AOI_WT = BBox(bbox = (8.629496, 47.421583, 8.884906,47.58301), 
              crs = CRS.WGS84)

Segment Area of Intrest based on resolution

In [6]:
AOI_ZH_segments = segment_aoi(aoi = AOI_ZH, resolution = 10, output = "grid")
# TODO: evtl max pixel als parameter für kleinflächigere Bilder

AOI has Dimensions: 3391px x 3673px (h x w)
AOI has been split into Grid with Dimensions: 3 x 3 (rows x cols)


Define Time Intervall and Max Cloud Coverage

In [7]:
print(AOI_ZH_segments[0])

[BBox(((8.396073, 47.46039533333333), (8.557649333333332, 47.562627)), crs=CRS('4326')), BBox(((8.557649333333332, 47.46039533333333), (8.719225666666665, 47.562627)), crs=CRS('4326')), BBox(((8.719225666666667, 47.46039533333333), (8.880802, 47.562627)), crs=CRS('4326'))]


In [ ]:
TIME_INTERVALL = "2024-11-29", "2024-11-29"
MAX_CC = 20

create Dictionary to Map AOI Segment ID's to respective BBoxes

In [76]:
AOI_BBOX_MAP = create_aoi_bbox_map(AOI_WT_segments)
print(AOI_BBOX_MAP)
#TODO: aktuell maximal grid von 99x99 möglich (2 stellige row bzw column id)

{'0000': BBox(((8.629496, 47.5022965), (8.757201, 47.58301)), crs=CRS('4326')), '0001': BBox(((8.757201, 47.5022965), (8.884906, 47.58301)), crs=CRS('4326')), '0100': BBox(((8.629496, 47.421583), (8.757201, 47.5022965)), crs=CRS('4326')), '0101': BBox(((8.757201, 47.421583), (8.884906, 47.5022965)), crs=CRS('4326'))}


### Catalog API

> **AOI_CATALOG**: Dictionary with Timestamps and CC of all recordings in specified Timerange for all AOI Segments

> **AOI_DF**: Dataframe with Timestamps and CC of all recordings in specified Timerange for all AOI Segments


In [ ]:
AOI_CATALOG = create_aoi_catalog(CATALOG, TIME_INTERVALL, AOI_BBOX_MAP, save_file = True)
print(AOI_CATALOG)

AOI_DF = create_aoi_catalog_df_old_data(AOI_CATALOG)
# TODO: verschiedene Datentypen
AOI_DF.head()

{'0000': {'2024-11-29T10:37:44Z': 18.88, '2024-11-26T10:27:47Z': 96.44, '2024-11-24T10:37:46Z': 96.39, '2024-11-19T10:37:45Z': 91.69, '2024-11-16T10:27:48Z': 33.13, '2024-11-14T10:37:41Z': 48.84, '2024-11-11T10:27:43Z': 70.84, '2024-11-09T10:37:43Z': 86.21, '2024-11-06T10:27:44Z': 57.57, '2024-11-04T10:37:42Z': 61.49, '2024-11-01T10:27:48Z': 35.75}, '0001': {'2024-11-29T10:37:44Z': 18.88, '2024-11-26T10:27:47Z': 96.44, '2024-11-24T10:37:46Z': 96.39, '2024-11-19T10:37:45Z': 91.69, '2024-11-16T10:27:48Z': 33.13, '2024-11-14T10:37:41Z': 48.84, '2024-11-11T10:27:43Z': 70.84, '2024-11-09T10:37:43Z': 86.21, '2024-11-06T10:27:44Z': 57.57, '2024-11-04T10:37:42Z': 61.49, '2024-11-01T10:27:48Z': 35.75}, '0100': {'2024-11-29T10:37:44Z': 18.88, '2024-11-26T10:27:47Z': 96.44, '2024-11-24T10:37:46Z': 96.39, '2024-11-19T10:37:45Z': 91.69, '2024-11-16T10:27:48Z': 33.13, '2024-11-14T10:37:41Z': 48.84, '2024-11-11T10:27:43Z': 70.84, '2024-11-09T10:37:43Z': 86.21, '2024-11-06T10:27:44Z': 57.57, '2024-11-

,AOI_ID,TS,CC,UID
0,0000,2024-11-29T10:37:44Z,18.88,0000_2024-11-29T10:37:44Z
1,0000,2024-11-26T10:27:47Z,96.44,0000_2024-11-26T10:27:47Z
2,0000,2024-11-24T10:37:46Z,96.39,0000_2024-11-24T10:37:46Z
3,0000,2024-11-19T10:37:45Z,91.69,0000_2024-11-19T10:37:45Z
4,0000,2024-11-16T10:27:48Z,33.13,0000_2024-11-16T10:27:48Z


### Process API

> **AOI_CATALOG_2**: Dictionary with Timestamps and CC from API and CC self calculated of all recordings in specified Timerange for all AOI Segments

> **AOI_DF_2**: DF für das obe

calculate and display actual cloud coverage

In [97]:
AOI_CATALOG_2 = calculate_cc(AOI_CATALOG, AOI_BBOX_MAP, resolution = 10, CONFIG = CONFIG)
AOI_DF_2 = create_aoi_df_2(AOI_CATALOG_2)
AOI_DF_2.head()


Calculating Cloud Coverage
Estimated Time: 106.0 seconds, 1.7746666666666664 minutes


,AOI_ID,Timestamp,CC_API,CC_CLC
0,0000,2024-11-29T10:37:44Z,18.88,90.812595
1,0000,2024-11-26T10:27:47Z,96.44,61.133050
2,0000,2024-11-24T10:37:46Z,96.39,99.492309
3,0000,2024-11-19T10:37:45Z,91.69,96.935868
4,0000,2024-11-16T10:27:48Z,33.13,93.466861


In [103]:
evalscript = load_evalscript("evalscripts/es_true_color.js")
resolution = 10
for aoi_id in AOI_CATALOG_2.keys():
    aoi_recordings = AOI_CATALOG_2[aoi_id]
    aoi_bbox = AOI_BBOX_MAP[aoi_id]
    for recording in aoi_recordings.keys():
        timestamp = recording
        img = get_img(evalscript = evalscript, timestamp = str(timestamp)[:10], bbox = aoi_bbox, resolution = resolution, CONFIG = CONFIG)
        image_name = aoi_id+"_"+str(timestamp)[:10]+".png"
        download_img(img = img, path = path_images, filename = image_name)


In [113]:
AOI_DF_2.head()

,AOI_ID,Timestamp,CC_API,CC_CLC,Image_Path
0,0000,2024-11-29T10:37:44Z,18.88,90.812595,images/0000_2024-11-29.png
1,0000,2024-11-26T10:27:47Z,96.44,61.133050,images/0000_2024-11-26.png
2,0000,2024-11-24T10:37:46Z,96.39,99.492309,images/0000_2024-11-24.png
3,0000,2024-11-19T10:37:45Z,91.69,96.935868,images/0000_2024-11-19.png
4,0000,2024-11-16T10:27:48Z,33.13,93.466861,images/0000_2024-11-16.png


In [128]:
AOI_DF_2["Image_Path"] = "images/"+AOI_DF_2['AOI_ID']+"_"+AOI_DF_2["Timestamp"].str.slice(0,10)+".png"
AOI_DF_2_OUT = AOI_DF_2[['AOI_ID', 'Timestamp', 'CC_API', 'CC_CLC', 'Image_Path']].copy()
#AOI_DF_2_OUT.head()
AOI_DF_2_OUT_Sorted = AOI_DF_2_OUT
AOI_DF_2_OUT_Sorted = AOI_DF_2_OUT_Sorted.sort_values(by = 'CC_API')
AOI_DF_2_OUT_Sorted.head()

,AOI_ID,Timestamp,CC_API,CC_CLC,Image_Path
0,0000,2024-11-29T10:37:44Z,18.88,90.812595,images/0000_2024-11-29.png
33,0101,2024-11-29T10:37:44Z,18.88,73.702608,images/0101_2024-11-29.png
22,0100,2024-11-29T10:37:44Z,18.88,70.466291,images/0100_2024-11-29.png
11,0001,2024-11-29T10:37:44Z,18.88,86.009618,images/0001_2024-11-29.png
37,0101,2024-11-16T10:27:48Z,33.13,63.549831,images/0101_2024-11-16.png


In [ ]:
import pandas as pd
from IPython.display import display, HTML
from PIL import Image
import io
import base64


def display_dataframe_with_images(dataframe):
    """Displays a pandas DataFrame with images from the 'Image_Path' column."""

    html_table = "<table><tr>"
    for col in dataframe.columns:
        html_table += f"<th>{col}</th>"
    html_table += "</tr>"

    for index, row in dataframe.iterrows():
        html_table += "<tr>"
        for col in dataframe.columns:
            if col == 'Image_Path':
                try:
                    # Open the image file
                    img = Image.open(row[col])

                    # Convert the image to base64
                    buffered = io.BytesIO()
                    img.save(buffered, format="PNG")  # Or another suitable format
                    img_str = base64.b64encode(buffered.getvalue()).decode()

                    # Embed the image in the HTML table
                    html_table += f"<td><img src='data:image/png;base64,{img_str}' width='400'></td>" #Adjust width as needed.
                except FileNotFoundError:
                    html_table += "<td>Image not found</td>"
                except Exception as e:
                    html_table += f"<td>Error loading image: {e}</td>"
            else:
                html_table += f"<td>{row[col]}</td>"
        html_table += "</tr>"
    html_table += "</table>"
    return html_table

# Call the function to display the DataFrame with images
def save_html_table(html_string, filename="output_table.html"):
    """Saves an HTML string to a file."""
    with open(filename, "w", encoding="utf-8") as f:
        f.write(html_string)
    print(f"HTML table saved to {filename}")

In [129]:
html_table = display_dataframe_with_images(AOI_DF_2_OUT_Sorted)
#display(HTML(html_table))
save_html_table(html_table) #saves to output_table.html.

HTML table saved to output_table.html


### Test Images

In [136]:
AOI_DF_2
test_aoi_bbox = AOI_BBOX_MAP["0000"]
test_ts = str("2024-11-29T10:37:44Z")[:10]
evalscript = load_evalscript("evalscripts/es_true_color.js")
test_img = get_img_lcc(evalscript,  test_ts, test_aoi_bbox, 10, CONFIG)
download_img(test_img, "images", "test_mosaickingorder.png")

In [ ]:
test_aoi_bbox = AOI_BBOX_MAP["0000"]
test_ts = str("2024-11-29T10:37:44Z")[:10]
evalscript = load_evalscript("evalscripts/es_built-up_binary_classifier.js")
test_img = get_img(evalscript,  test_ts, test_aoi_bbox, 10, CONFIG)
download_img(test_img, "images", "test_built_up.png")

In [ ]:
test_aoi_bbox = AOI_BBOX_MAP["0000"]
test_ts = str("2024-11-29T10:37:44Z")[:10]
evalscript = load_evalscript("evalscripts/es_built-up_binary_classifier.js")
test_img = get_img(evalscript,  test_ts, test_aoi_bbox, 10, CONFIG)
download_img(test_img, "images", "test_built_up.png")

In [100]:
AOI_DF_2_F = filter_aoi_df(AOI_DF_2, MAX_CC)
AOI_DF_2_F.head()

removing entries with cloud coverage > 20 
> found 43 entries to remove
> filtered dataframe has 1 entries


,AOI_ID,Timestamp,CC_API,CC_CLC
43,0101,2024-11-01T10:27:48Z,35.75,8.652329
